In [1]:
import pandas as pd
import ast
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import pickle
from scipy import sparse
import numpy as np
import gzip
import shutil
import os

def preprocess_tmdb_data(movies_path='tmdb_5000_movies.csv.zip', credits_path='tmdb_5000_credits.csv.zip'):
    """
    Optimized TMDB data preprocessing with size reduction
    Returns: (movies_df, cosine_sim, indices)
    """
    
    # Load and merge datasets 
    try:
        movies = pd.read_csv(movies_path)
        credits = pd.read_csv(credits_path)
    except FileNotFoundError as e:
        print(f"Error loading files: {e}")
        raise

    movies = movies.merge(credits, left_on='id', right_on='movie_id')

    # Helper functions 
    def safe_json_extract(text, field='name'):
        try:
            lst = ast.literal_eval(text) if isinstance(text, str) else text
            return ' '.join([str(i.get(field, '')) for i in lst]) if lst else ''
        except (ValueError, SyntaxError, AttributeError):
            return ''

    def get_director(crew_json):
        try:
            crew = ast.literal_eval(crew_json) if isinstance(crew_json, str) else crew_json
            directors = [m['name'] for m in crew if m.get('job') == 'Director']
            return directors[0] if directors else ''
        except (ValueError, SyntaxError, AttributeError, TypeError):
            return ''
    
    def get_top_cast(cast_json):
        try:
            cast = ast.literal_eval(cast_json) if isinstance(cast_json, str) else cast_json
            return ' '.join([m['name'] for m in cast[:3]])
        except (ValueError, SyntaxError, AttributeError, TypeError):
            return ''
    
    def get_detailed_cast(cast_json):
        try:
            cast = ast.literal_eval(cast_json) if isinstance(cast_json, str) else cast_json
            return [{
                'name': m.get('name', ''),
                'character': m.get('character', ''),
                'profile_path': m.get('profile_path', '')
            } for m in cast[:5]]  # Top 5 cast members
        except (ValueError, SyntaxError, AttributeError, TypeError):
            return []

    # Process columns 
    movies['genres_str'] = movies['genres'].apply(lambda x: safe_json_extract(x))
    movies['keywords_str'] = movies['keywords'].apply(lambda x: safe_json_extract(x))
    movies['director'] = movies['crew'].apply(get_director)
    movies['top_cast'] = movies['cast'].apply(get_top_cast)
    movies['detailed_cast'] = movies['cast'].apply(get_detailed_cast)
    movies['genres_list'] = movies['genres'].apply(
        lambda x: [i['name'] for i in ast.literal_eval(x)] if isinstance(x, str) else []
    )
    movies['combined_features'] = (
        movies['genres_str'] + ' ' +
        movies['keywords_str'] + ' ' +
        movies['director'] + ' ' +
        movies['top_cast'] + ' ' +
        movies['overview'].fillna('')
    )

    # Fill missing values 
    movies['runtime'] = movies['runtime'].fillna(0).astype(int) if 'runtime' in movies.columns else 0
    movies['vote_average'] = movies['vote_average'].fillna(0) if 'vote_average' in movies.columns else 0
    movies['poster_path'] = movies['poster_path'].fillna('') if 'poster_path' in movies.columns else ''
    movies['release_date'] = movies['release_date'].fillna('') if 'release_date' in movies.columns else ''
    
    # ===== OPTIMIZED SIMILARITY MATRIX GENERATION =====
    tfidf = TfidfVectorizer(
        stop_words='english',
        max_features=8000,      # Reduced features
        ngram_range=(1, 2),     # Better coverage , consider both unigram and bygram word
        min_df=2,               # Filter rare terms less than 2 times
        max_df=0.85             # Filter common terms more than 85% document
    )
    
    tfidf_matrix = tfidf.fit_transform(movies['combined_features'])
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
    
    # Optimization 1: Convert to float32
    cosine_sim = cosine_sim.astype(np.float32)
    
    # Optimization 2: Convert to sparse format
    if not sparse.issparse(cosine_sim):
        cosine_sim = sparse.csr_matrix(cosine_sim)
    
    # Create indices 
    indices = pd.Series(movies.index, index=movies['title_y']).drop_duplicates()

    # ===== OPTIMIZED SAVING =====
    # 1. Save with protocol 4
    with open('cosine_sim.pkl', 'wb') as f:
        pickle.dump(cosine_sim, f, protocol=4)
    
    # 2. Compress the file
    with open('cosine_sim.pkl', 'rb') as f_in:
        with gzip.open('cosine_sim.pkl.gz', 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
    
    # Save other files 
    movies.to_pickle('processed_movies.pkl')
    with open('indices.pkl', 'wb') as f:
        pickle.dump(indices, f)
    
    print("Optimized preprocessing complete!")
    print(f"Final cosine_sim size: {os.path.getsize('cosine_sim.pkl.gz')/1024/1024:.2f}MB")
    
    return movies, cosine_sim, indices

if __name__ == '__main__':
    df, cos_sim, index_map = preprocess_tmdb_data()

Optimized preprocessing complete!
Final cosine_sim size: 69.81MB


In [ ]:
if __name__ == '__main__':
    # Example usage
    try:
        df, cos_sim, index_map = preprocess_tmdb_data()
        print("\nSample movie data:")
        print(f"Title: {df['title_y'].iloc[0]}")
        print(f"Poster path: {df['poster_path'].iloc[0]}")
        
        # Check cast data
        first_movie_cast = df['detailed_cast'].iloc[0]
        if first_movie_cast:
            print("\nFirst cast member details:")
            print(f"Name: {first_movie_cast[0]['name']}")
            print(f"Character: {first_movie_cast[0]['character']}")
            print(f"Profile path: {first_movie_cast[0]['profile_path']}")
        else:
            print("No cast data available for first movie")
            
        # Analyze poster path availability
        valid_posters = df['poster_path'].apply(lambda x: bool(x and str(x).strip())).sum()
        total_movies = len(df)
        print(f"\nPoster Path Analysis:")
        print(f"Movies with valid poster paths: {valid_posters}/{total_movies} ({valid_posters/total_movies:.1%})")
        
    except Exception as e:
        print(f"Error in preprocessing: {e}")


In [4]:
!pip install numpy

In [7]:
print('pandas', pd.__version__)

pandas 2.2.3


In [11]:
!pip install scikit-learn

In [13]:
print('sklearn',sklearn.__version__)

NameError: name 'sklearn' is not defined

In [2]:
pip install numpy 


Note: you may need to restart the kernel to use updated packages.
